# Data Science sa Cloud: Ang "Azure ML SDK" na paraan

## Panimula

Sa notebook na ito, matututuhan natin kung paano gamitin ang Azure ML SDK upang mag-train, mag-deploy, at gumamit ng isang modelo sa pamamagitan ng Azure ML.

Mga kinakailangan:
1. Nakagawa ka na ng Azure ML workspace.
2. Na-load mo na ang [Heart Failure dataset](https://www.kaggle.com/andrewmvd/heart-failure-clinical-data) sa Azure ML.
3. Na-upload mo na ang notebook na ito sa Azure ML Studio.

Ang mga susunod na hakbang ay:

1. Gumawa ng isang Experiment sa isang umiiral na Workspace.
2. Gumawa ng Compute cluster.
3. I-load ang dataset.
4. I-configure ang AutoML gamit ang AutoMLConfig.
5. Patakbuhin ang AutoML experiment.
6. Suriin ang mga resulta at kunin ang pinakamahusay na modelo.
7. Irehistro ang pinakamahusay na modelo.
8. I-deploy ang pinakamahusay na modelo.
9. Gamitin ang endpoint.

## Mga import na partikular sa Azure Machine Learning SDK


In [ ]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails
from azureml.core.model import InferenceConfig, Model
from azureml.core.webservice import AciWebservice

## I-initialize ang Workspace
I-initialize ang isang workspace object mula sa naka-save na configuration. Siguraduhing naroroon ang config file sa .\config.json


In [ ]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

## Gumawa ng Azure ML na eksperimento

Gumawa tayo ng eksperimento na pinangalanang 'aml-experiment' sa workspace na kakalunsad lang natin.


In [ ]:
experiment_name = 'aml-experiment'
experiment = Experiment(ws, experiment_name)
experiment

## Gumawa ng Compute Cluster
Kailangan mong gumawa ng [compute target](https://docs.microsoft.com/azure/machine-learning/concept-azure-machine-learning-architecture#compute-target) para sa iyong AutoML run.


In [ ]:
aml_name = "heart-f-cluster"
try:
    aml_compute = AmlCompute(ws, aml_name)
    print('Found existing AML compute context.')
except:
    print('Creating new AML compute context.')
    aml_config = AmlCompute.provisioning_configuration(vm_size = "Standard_D2_v2", min_nodes=1, max_nodes=3)
    aml_compute = AmlCompute.create(ws, name = aml_name, provisioning_configuration = aml_config)
    aml_compute.wait_for_completion(show_output = True)

cts = ws.compute_targets
compute_target = cts[aml_name]

## Datos  
Siguraduhing na-upload mo na ang dataset sa Azure ML at ang susi ay kapareho ng pangalan ng dataset.


In [ ]:
key = 'heart-failure-records'
dataset = ws.datasets[key]
df = dataset.to_pandas_dataframe()
df.describe()

In [ ]:
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 3,
    "primary_metric" : 'AUC_weighted'
}

automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="DEATH_EVENT",
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [ ]:
remote_run = experiment.submit(automl_config)

In [ ]:
RunDetails(remote_run).show()

In [ ]:
best_run, fitted_model = remote_run.get_output()

In [ ]:
best_run.get_properties()

In [ ]:
model_name = best_run.properties['model_name']
script_file_name = 'inference/score.py'
best_run.download_file('outputs/scoring_file_v_1_0_0.py', 'inference/score.py')
description = "aml heart failure project sdk"
model = best_run.register_model(model_name = model_name,
                                description = description,
                                tags = None)

## I-deploy ang Pinakamahusay na Modelo

Patakbuhin ang sumusunod na code upang i-deploy ang pinakamahusay na modelo. Makikita mo ang estado ng deployment sa Azure ML portal. Ang hakbang na ito ay maaaring tumagal ng ilang minuto.


In [ ]:
inference_config = InferenceConfig(entry_script=script_file_name, environment=best_run.get_environment())

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1,
                                               memory_gb = 1,
                                               tags = {'type': "automl-heart-failure-prediction"},
                                               description = 'Sample service for AutoML Heart Failure Prediction')

aci_service_name = 'automl-hf-sdk'
aci_service = Model.deploy(ws, aci_service_name, [model], inference_config, aciconfig)
aci_service.wait_for_deployment(True)
print(aci_service.state)

## Gamitin ang Endpoint
Maaari kang magdagdag ng mga input sa sumusunod na halimbawa ng input.


In [ ]:
data = {
    "data":
    [
        {
            'age': "60",
            'anaemia': "false",
            'creatinine_phosphokinase': "500",
            'diabetes': "false",
            'ejection_fraction': "38",
            'high_blood_pressure': "false",
            'platelets': "260000",
            'serum_creatinine': "1.40",
            'serum_sodium': "137",
            'sex': "false",
            'smoking': "false",
            'time': "130",
        },
    ],
}

test_sample = str.encode(json.dumps(data))

In [ ]:
response = aci_service.run(input_data=test_sample)
response


---

**Paunawa**:  
Ang dokumentong ito ay isinalin gamit ang AI translation service na [Co-op Translator](https://github.com/Azure/co-op-translator). Bagama't sinisikap naming maging tumpak, pakitandaan na ang mga awtomatikong pagsasalin ay maaaring maglaman ng mga pagkakamali o hindi pagkakatugma. Ang orihinal na dokumento sa orihinal nitong wika ang dapat ituring na opisyal na sanggunian. Para sa mahalagang impormasyon, inirerekomenda ang propesyonal na pagsasalin ng tao. Hindi kami mananagot para sa anumang hindi pagkakaunawaan o maling interpretasyon na maaaring magmula sa paggamit ng pagsasaling ito.
